## 2.10 종합 예제
스파크 DataFrame의 스키마 정보를 알아내는 스키마 추론(schema inference)과 파일의 첫 Row을 헤더로 지정하는 옵션을 사용하여 csv파일을 읽는다.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
flightData2015 = spark \
    .read \
    .option('inferSchema', 'true') \
    .option('header', 'true') \
    .csv('./data/flight-data/csv/2015-summary.csv') \

스칼라와 파이썬에서 사용하는 DataFrame은 불특정 다수의 로우와 컬럼을 가진다. 로우의 수를 알 수 없는 이유는 지연 연산형태의 프랜스포메이션이기 때문이며, 스파크는 각 컬럼의 데이터 타입을 추론하기 위해 적은 양의 데이터를 읽는다. DataFrame의 take 액션을 호출하면 이제서야 결과를 확인할 수 있다.

In [3]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

정수 데이터 타입인 count 컬럼을 기준으로 데이터를 정렬하는 트랜스포메이션을 추가하였다. sort는 트랜스포메이션이기 때문에 데이터에 변화가 일어나지는 않지만 explain 메서드를 통해 스파크 쿼리의 실행계획을 확인할 수 있다.

In [4]:
flightData2015.sort("count").explain() # 200개의 셔플 파티션을 생성한 것을 아래 확인할 수 있다.

== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/SparkDefinitiveGuide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


액션을 호출하면 트랜스포메이션 실행 계획을 시작한다. 액션을 시작하기 전 셔플의 출력 파티션 수를 줄인다.

In [5]:
spark.conf.set('spark.sql.shuffle.partitions', '5')
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

### 2.10.1 DataFrame과 SQL
스파크 SQL을 사용하면 모든 DataFrame을 테이블이나 뷰(임시 테이블)로 등록한 후 SQL 쿼리를 사용할 수 있다. createOrReplaceTempView 메서드를 사용한다.

In [6]:
flightData2015.createOrReplaceTempView('flight_data_2015')

DataFrame과 SQL 중 간편한 방식으로 트랜스포메이션을 지정할 수 있다. 같은 목적으로 두 종류의 실행 계획을 비교해보면 동일한 기본 실행 계획으로 컴파일되었다.

In [7]:
query = """
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
"""
sqlWay = spark.sql(query)
dataFramesWay = flightData2015.groupBy('DEST_COUNTRY_NAME').count()

sqlWay.explain()
dataFramesWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/SparkDefinitiveGuide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/SparkDefinitiveGuide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAM

특정 위치를 왕래하는 최대 비행 횟수를 구하기 위해 max함수를 활용한다. max 함수는 필터링을 수행해 단일 로우를 결과로 반환하는 트랜스포메이션이다.

In [8]:
# SQL
spark.sql('SELECT max(count) from flight_data_2015').take(1)

# Python
from pyspark.sql.functions import max
flightData2015.select(max('count')).take(1)

[Row(max(count)=370002)]

상위 5개의 도착 국가를 찾아내는 복잡한 코드

In [13]:
# SQL
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

# Python
from pyspark.sql.functions import desc

flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "destination_total") \
    .sort(desc("destination_total")) \
    .limit(5) \
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [14]:
flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "destination_total") \
    .sort(desc("destination_total")) \
    .limit(5) \
    .explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#145L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#145L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/SparkDefinitiveGuide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


(1)은 파일을 읽음, 그 다음 (1~2)는 groupBy, sum()을 실행, 마지막으로는 정렬과 limit(5)를 실행하는 순서임  